In [6]:
import pandas as pd
import numpy as np
import os
import tifffile as tiff
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the dataset
df = pd.read_csv("datasets/train.csv")

print(f'Number of features: {df.shape[1]}')
print(f'Number of instances: {df.shape[0]}')
df.head()

# Preprocess numerical data
numerical_features = df[['elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', 'incidence_zenith']]

# Numerical data scaling
scaler = StandardScaler()
numerical_features = scaler.fit_transform(numerical_features)

# Function to load and preprocess image data
def load_and_preprocess_image(filepath):
    img = tiff.imread(filepath)
    img_array = np.array(img)
    img_array = img_array / 65535.0   # Normalize pixel values
    return img_array

# Load image data
image_data = np.array([load_and_preprocess_image(os.path.join('./train/', filename)) for filename in df['file_name_l1']])

# Target variable
target = df['value_550'].values

# Split data into training, validation, and testing sets
X_train_num, X_temp_num, X_train_img, X_temp_img, y_train, y_temp = train_test_split(numerical_features, image_data, target, test_size=0.3, random_state=42)
X_val_num, X_test_num, X_val_img, X_test_img, y_val, y_test = train_test_split(X_temp_num, X_temp_img, y_temp, test_size=0.5, random_state=42)

# Define the CNN and dense model
class AOTModel:
    def __init__(self, image_shape=(19, 19, 13), num_numerical_features=7):
        # Image processing Neural Network
        self.image_input = Input(shape=image_shape)
        image_processing_network = Conv2D(32, (3, 3), activation='relu')(self.image_input)
        image_processing_network = BatchNormalization()(image_processing_network)
        image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        image_processing_network = Dropout(0.25)(image_processing_network)

        image_processing_network = Conv2D(64, (3, 3), activation='relu')(image_processing_network)
        image_processing_network = BatchNormalization()(image_processing_network)
        image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        image_processing_network = Dropout(0.25)(image_processing_network)

        image_processing_network = Flatten()(image_processing_network)
        image_processing_network = Dense(64, activation='relu')(image_processing_network)
        image_processing_network = Dropout(0.5)(image_processing_network)

        # Numerical processing Neural Network
        self.numerical_input = Input(shape=(num_numerical_features,))
        numerical_processing_network = Dense(64, activation='relu')(self.numerical_input)
        numerical_processing_network = BatchNormalization()(numerical_processing_network)
        numerical_processing_network = Dropout(0.5)(numerical_processing_network)
        
        # Concatenation of both networks
        aot_network = Concatenate()([image_processing_network, numerical_processing_network])
        aot_network = Dense(64, activation='relu')(aot_network)
        aot_network = Dropout(0.5)(aot_network)
        aot_network = Dense(1)(aot_network)

        self.aot_network_arquitecture = aot_network
        del image_processing_network, numerical_processing_network, aot_network

    def model(self):
        model = Model(inputs= [self.image_input, self.numerical_input], outputs=self.aot_network_arquitecture)
        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
        return model

# Instantiate the model
model = AOTModel()
model = model.model()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Data augmentation
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=10)

# Train the model
history = model.fit(
    [X_train_img, X_train_num], y_train,
    validation_data=([X_val_img, X_val_num], y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

model.save('aot_model.h5')

# Evaluate the model
val_loss, val_mae = model.evaluate([X_val_img, X_val_num], y_val)
print(f'Validation MAE: {val_mae}')

# Evaluate on test set
test_loss, test_mae = model.evaluate([X_test_img, X_test_num], y_test)
print(f'Test MAE: {test_mae}')

Number of features: 10
Number of instances: 10438
Epoch 1/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 30s 62ms/step - loss: 2.0562 - mae: 1.0101 - val_loss: 0.0380 - val_mae: 0.1111 - learning_rate: 0.0010
Epoch 2/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2380 - mae: 0.3521 - val_loss: 0.0343 - val_mae: 0.1043 - learning_rate: 0.0010
Epoch 3/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0835 - mae: 0.1892 - val_loss: 0.0331 - val_mae: 0.0974 - learning_rate: 0.0010
Epoch 4/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0449 - mae: 0.1302 - val_loss: 0.0296 - val_mae: 0.0915 - learning_rate: 0.0010
Epoch 5/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0354 - mae: 0.1142 - val_loss: 0.0279 - val_mae: 0.0951 - learning_rate: 0.0010
Epoch 6/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0316 - mae: 0.1081 - val_loss: 0.0280 - val_mae: 0.0946 - learning_rate: 0.0010
Epoch 7/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0314 - mae: 0.1050 - val_lo

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0145 - mae: 0.0774
Validation MAE: 0.07634811103343964
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0120 - mae: 0.0724
Test MAE: 0.0765424445271492


In [8]:
# Load the saved model
model = load_model('aot_model.h5')

# Load the new dataset
new_df = pd.read_csv("datasets/test.csv")

# Preprocess numerical data
new_numerical_features = new_df[['elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', 'incidence_zenith']]
new_numerical_features = scaler.transform(new_numerical_features)

# Load and preprocess new image data
new_image_data = np.array([load_and_preprocess_image(os.path.join('./test/', filename)) for filename in new_df['file_name_l1']])

# Predict values for the new data
predictions = model.predict([new_image_data, new_numerical_features])

# Save the predictions to a CSV file
results = pd.DataFrame({
    'id': new_df['id'],
    'value_550': predictions.flatten()
})
results.to_csv('predictions.csv', index=False)

print(results.head())   

85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
   id  value_550
0   3   0.148174
1  25   0.177376
2  26   0.099280
3  27   0.114423
4  29   0.114808
